## DE Pipeline Project
### Chris Joyce

In [ ]:
%load_ext sql
%sql sqlite:///ticker_tot.db
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
import json
import requests
import pandas as pd
import sqlite3
import re
import time
import shutil
import os
from datetime import datetime
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
db_complete = 'ticker_tot.db'
db_path = '/content/drive/MyDrive/Colab/ticker_tot.db'
shutil.copy(db_complete, db_path)
API_KEY = userdata.get('polygon')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'COST', 'MA', 'V']
timestamp = '2023-08-31'
timespan = 'day'
adjusted = 'true'
series_type = 'close'
order = 'desc'
rsi_res = {}
oc_res = {}

In [ ]:
for ticker in tickers:
    rsi_url = f"https://api.polygon.io/v1/indicators/rsi/{ticker}?timestamp={timestamp}&timespan={timespan}&adjusted={adjusted}&series_type={series_type}&order={order}&apiKey={API_KEY}"
    rsi_response = requests.get(rsi_url)
    if rsi_response.status_code == 200:
        rsi_res[ticker] = rsi_response.json()
        print(f"Response for {ticker}:")
        print(rsi_res[ticker])
    else:
        print(f"Failed to retrieve data for {ticker}")
    time.sleep(12)

Response for AAPL:
{'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/1687320000000/1693526400000?limit=75&sort=desc'}, 'values': [{'timestamp': 1693454400000, 'value': 59.833966914775324}]}, 'status': 'OK', 'request_id': '4e78171e136e2117f60d7aed26b64a79'}
Response for GOOGL:
{'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/GOOGL/range/1/day/1687320000000/1693526400000?limit=75&sort=desc'}, 'values': [{'timestamp': 1693454400000, 'value': 66.11385475718521}]}, 'status': 'OK', 'request_id': 'f2c86c626e1f974fcd93e090a021a62c'}
Response for MSFT:
{'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/MSFT/range/1/day/1687320000000/1693526400000?limit=75&sort=desc'}, 'values': [{'timestamp': 1693454400000, 'value': 50.87298317199052}]}, 'status': 'OK', 'request_id': '72069d26caffaa5b45d5d7f61997b80f'}
Response for AMZN:
{'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/AMZN/range/1/

In [ ]:
for ticker in tickers:
    oc_url = f"https://api.polygon.io/v1/open-close/{ticker}/{timestamp}?adjusted=true&apiKey={API_KEY}"
    oc_response = requests.get(oc_url)
    if oc_response.status_code == 200:
        oc_res[ticker] = oc_response.json()
        print(f"Response for {ticker}:")
        print(oc_res[ticker])
    else:
        print(f"Failed to retrieve data for {ticker}")
    time.sleep(12)

Response for AAPL:
{'status': 'OK', 'from': '2023-08-31', 'symbol': 'AAPL', 'open': 187.84, 'high': 189.12, 'low': 187.48, 'close': 187.87, 'volume': 60794467.0, 'afterHours': 188.11, 'preMarket': 187.8}
Response for GOOGL:
{'status': 'OK', 'from': '2023-08-31', 'symbol': 'GOOGL', 'open': 136.01, 'high': 138, 'low': 135.79, 'close': 136.17, 'volume': 30053800.0, 'afterHours': 136.42, 'preMarket': 135.91}
Response for MSFT:
{'status': 'OK', 'from': '2023-08-31', 'symbol': 'MSFT', 'open': 329.2, 'high': 330.9099, 'low': 326.78, 'close': 327.76, 'volume': 26403154.0, 'afterHours': 328.31, 'preMarket': 329.59}
Response for AMZN:
{'status': 'OK', 'from': '2023-08-31', 'symbol': 'AMZN', 'open': 135.06, 'high': 138.7885, 'low': 135, 'close': 138.01, 'volume': 58431314.0, 'afterHours': 138.4, 'preMarket': 135.12}
Response for TSLA:
{'status': 'OK', 'from': '2023-08-31', 'symbol': 'TSLA', 'open': 255.98, 'high': 261.18, 'low': 255.05, 'close': 258.08, 'volume': 108861698.0, 'afterHours': 257.96

In [ ]:
rsi_res

{'AAPL': {'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/1687320000000/1693526400000?limit=75&sort=desc'},
   'values': [{'timestamp': 1693454400000, 'value': 59.833966914775324}]},
  'status': 'OK',
  'request_id': '4e78171e136e2117f60d7aed26b64a79'},
 'GOOGL': {'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/GOOGL/range/1/day/1687320000000/1693526400000?limit=75&sort=desc'},
   'values': [{'timestamp': 1693454400000, 'value': 66.11385475718521}]},
  'status': 'OK',
  'request_id': 'f2c86c626e1f974fcd93e090a021a62c'},
 'MSFT': {'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/MSFT/range/1/day/1687320000000/1693526400000?limit=75&sort=desc'},
   'values': [{'timestamp': 1693454400000, 'value': 50.87298317199052}]},
  'status': 'OK',
  'request_id': '72069d26caffaa5b45d5d7f61997b80f'},
 'AMZN': {'results': {'underlying': {'url': 'https://api.polygon.io/v2/aggs/ticker/AMZN/range/1/day/168732000000

In [ ]:
oc_res

{'AAPL': {'status': 'OK',
  'from': '2023-08-31',
  'symbol': 'AAPL',
  'open': 187.84,
  'high': 189.12,
  'low': 187.48,
  'close': 187.87,
  'volume': 60794467.0,
  'afterHours': 188.11,
  'preMarket': 187.8},
 'GOOGL': {'status': 'OK',
  'from': '2023-08-31',
  'symbol': 'GOOGL',
  'open': 136.01,
  'high': 138,
  'low': 135.79,
  'close': 136.17,
  'volume': 30053800.0,
  'afterHours': 136.42,
  'preMarket': 135.91},
 'MSFT': {'status': 'OK',
  'from': '2023-08-31',
  'symbol': 'MSFT',
  'open': 329.2,
  'high': 330.9099,
  'low': 326.78,
  'close': 327.76,
  'volume': 26403154.0,
  'afterHours': 328.31,
  'preMarket': 329.59},
 'AMZN': {'status': 'OK',
  'from': '2023-08-31',
  'symbol': 'AMZN',
  'open': 135.06,
  'high': 138.7885,
  'low': 135,
  'close': 138.01,
  'volume': 58431314.0,
  'afterHours': 138.4,
  'preMarket': 135.12},
 'TSLA': {'status': 'OK',
  'from': '2023-08-31',
  'symbol': 'TSLA',
  'open': 255.98,
  'high': 261.18,
  'low': 255.05,
  'close': 258.08,
  'vo

In [ ]:
combi_values = []

In [ ]:
for ticker, response in rsi_res.items():
    if 'results' in response and 'values' in response['results']:
        for value in response['results']['values']:
            value['ticker'] = ticker
            if ticker in oc_res:
              value.update(oc_res[ticker])
            combi_values.append(value)

In [ ]:
combi_res_df = pd.json_normalize(combi_values)
combi_res_df

,timestamp,value,ticker,status,from,symbol,open,high,low,close,volume,afterHours,preMarket
0,1693454400000,59.833967,AAPL,OK,2023-08-31,AAPL,187.84,189.1200,187.48,187.87,60794467.0,188.11,187.80
1,1693454400000,66.113855,GOOGL,OK,2023-08-31,GOOGL,136.01,138.0000,135.79,136.17,30053800.0,136.42,135.91
2,1693454400000,50.872983,MSFT,OK,2023-08-31,MSFT,329.20,330.9099,326.78,327.76,26403154.0,328.31,329.59
3,1693454400000,58.361861,AMZN,OK,2023-08-31,AMZN,135.06,138.7885,135.00,138.01,58431314.0,138.40,135.12
4,1693454400000,57.915162,TSLA,OK,2023-08-31,TSLA,255.98,261.1800,255.05,258.08,108861698.0,257.96,255.29
5,1693454400000,53.072991,COST,OK,2023-08-31,COST,540.00,552.2000,540.00,549.28,2086507.0,549.99,542.22
6,1693454400000,67.687616,MA,OK,2023-08-31,MA,414.21,416.2900,411.78,412.64,2535134.0,412.24,415.70
7,1693454400000,62.897979,V,OK,2023-08-31,V,245.59,248.0200,245.45,245.68,5532610.0,246.69,247.50


In [ ]:
date_match = re.search(r'timestamp=(\d{4}-\d{2}-\d{2})', rsi_url)
if date_match:
    date = date_match.group(1)
    combi_res_df['date'] = date

In [ ]:
combi_res_df = combi_res_df.rename(columns = {'value' : 'rsi_value'})
combi_res_df = combi_res_df.drop(columns=['timestamp', 'status', 'open', 'high', 'low', 'close', 'volume', 'symbol', 'from'])

In [ ]:
combi_res_df

,rsi_value,ticker,afterHours,preMarket,date
0,59.833967,AAPL,188.11,187.80,2023-08-31
1,66.113855,GOOGL,136.42,135.91,2023-08-31
2,50.872983,MSFT,328.31,329.59,2023-08-31
3,58.361861,AMZN,138.40,135.12,2023-08-31
4,57.915162,TSLA,257.96,255.29,2023-08-31
5,53.072991,COST,549.99,542.22,2023-08-31
6,67.687616,MA,412.24,415.70,2023-08-31
7,62.897979,V,246.69,247.50,2023-08-31


In [ ]:
tech_group_df = pd.read_csv('/content/drive/MyDrive/Colab/techs_2019_2024.csv')
costco_df = pd.read_csv('/content/drive/MyDrive/Colab/COST_stock_data.csv')
mv_df = pd.read_csv('/content/drive/MyDrive/Colab/MVR.csv')

In [ ]:
tech_group_df.columns = tech_group_df.columns.str.lower()
tech_group_df = tech_group_df.rename(columns = {'adj close' : 'adjclose'})
#tech_group_df.head(10)

In [ ]:
costco_df = costco_df.rename(columns = {'Unnamed: 0' : 'date'})
#costco_df.head(10)

In [ ]:
mv_df = mv_df.rename(columns = {'Adj Close_M' : 'Adj_Close_M', 'Adj Close_V' : 'Adj_Close_V'})
m_df = mv_df[['Date', 'Open_M', 'High_M', 'Low_M', 'Close_M', 'Adj_Close_M', 'Volume_M']].copy()
m_df['ticker'] = 'MA'
m_df = m_df.rename(columns = {'Open_M' : 'Open', 'High_M' : 'High', 'Low_M' : 'Low', 'Close_M' : 'Close', 'Adj_Close_M' : 'Adjclose', 'Volume_M' : 'Volume'})
v_df = mv_df[['Date', 'Open_V', 'High_V', 'Low_V', 'Close_V', 'Adj_Close_V', 'Volume_V']].copy()
v_df['ticker'] = 'V'
v_df = v_df.rename(columns = {'Open_V' : 'Open', 'High_V' : 'High', 'Low_V' : 'Low', 'Close_V' : 'Close', 'Adj_Close_V' : 'Adjclose', 'Volume_V' : 'Volume'})
m_df.columns = m_df.columns.str.lower()
v_df.columns = v_df.columns.str.lower()
#m_df.head(10)

In [ ]:
conn = sqlite3.connect('ticker_tot.db')
cursor = conn.cursor()

In [ ]:
%sql drop table if exists ticker_flats;

""


In [ ]:
create_tf = ''' create table ticker_flats (
    date OBJECT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    adjclose REAL,
    volume INTEGER,
    ticker OBJECT
);
'''

conn.execute(create_tf)
conn.commit()

In [ ]:
#%sql drop table if exists ticker_api;

In [ ]:
create_ta = ''' create table ticker_api (
    date OBJECT,
    ticker OBJECT,
    rsi_value REAL,
    afterHours REAL,
    preMarket REAL

);
'''

conn.execute(create_ta)
conn.commit()

In [ ]:
tech_group_df.to_sql('ticker_flats', con=conn, if_exists='append', index=False)

6290

In [ ]:
costco_df.to_sql('ticker_flats', con=conn, if_exists='append', index=False)

9564

In [ ]:
m_df.to_sql('ticker_flats', con=conn, if_exists='append', index=False)

4047

In [ ]:
v_df.to_sql('ticker_flats', con=conn, if_exists='append', index=False)

4047

In [ ]:
combi_res_df.to_sql('ticker_api', con=conn, if_exists='append', index=False)

8

In [ ]:
%sql select * from ticker_flats where date = '2023-08-02';

,date,open,high,low,close,adjclose,volume,ticker
0,2023-08-02,195.039993,195.179993,191.850006,192.580002,191.562668,50389300,AAPL
1,2023-08-02,333.630005,333.630005,326.359985,327.500000,324.957947,27761300,MSFT
2,2023-08-02,130.149994,130.229996,126.820000,128.210007,128.210007,51027600,AMZN
3,2023-08-02,129.449997,130.089996,127.559998,128.380005,128.232834,26273300,GOOGL
4,2023-08-02,255.570007,259.519989,250.490005,254.110001,254.110001,101752900,TSLA
5,2023-08-02,552.260010,556.630005,551.020020,552.349976,536.411865,1635000,COST
6,2023-08-02,393.880005,394.209991,389.579987,390.019989,388.304718,2410600,MA
7,2023-08-02,238.630005,238.789993,236.789993,237.559998,235.733719,4868200,V


In [ ]:
%sql select * from ticker_api;

,date,ticker,rsi_value,afterHours,preMarket
0,2023-08-01,AAPL,63.309440,195.30,196.35
1,2023-08-01,GOOGL,65.855486,130.50,132.55
2,2023-08-01,MSFT,47.808717,334.39,335.53
3,2023-08-01,AMZN,54.698728,131.26,133.57
4,2023-08-01,TSLA,49.163105,258.73,267.38
...,...,...,...,...,...
179,2023-08-31,AMZN,58.361861,138.40,135.12
180,2023-08-31,TSLA,57.915162,257.96,255.29
181,2023-08-31,COST,53.072991,549.99,542.22
182,2023-08-31,MA,67.687616,412.24,415.70


In [ ]:
%sql drop table if exists ta_tf;

""


In [ ]:
create_ta_tf = """
create table ta_tf as
select
    ta.ticker,
    ta.date,
    ta.rsi_value,
    ta.afterHours,
    ta.preMarket,
    tf.open,
    tf.high,
    tf.low,
    tf.close,
    tf.adjclose,
    tf.volume
from
    ticker_api ta
left join
    ticker_flats tf
on
    ta.ticker = tf.ticker AND ta.date = tf.date;
"""

conn.execute(create_ta_tf)
conn.commit()

In [ ]:
mod_ta_tf = '''alter table ta_tf add column closed_volatility real;
'''
update_ta_tf = ''' update ta_tf
set closed_volatility = preMarket - afterHours;
'''

conn.execute(mod_ta_tf)
conn.execute(update_ta_tf)
conn.commit()

In [ ]:
mod_ta_tf1 = '''alter table ta_tf add column open_volatility real;
'''
update_ta_tf1 = ''' update ta_tf
set open_volatility = open - close;
'''

conn.execute(mod_ta_tf1)
conn.execute(update_ta_tf1)
conn.commit()

In [ ]:
mod_ta_tf2 = '''alter table ta_tf add column rsi_weighted real;
'''
update_ta_tf2 = ''' update ta_tf
set rsi_weighted = volume * rsi_value;
'''

conn.execute(mod_ta_tf2)
conn.execute(update_ta_tf2)
conn.commit()

In [ ]:
mod_ta_tf3 = '''alter table ta_tf add column rsi_indicator boolean;
'''
update_ta_tf3 = '''update ta_tf
set rsi_indicator = case when rsi_value > 70 then 'overbought' when rsi_value < 30 then 'oversold' else 'neutral' end;
'''

conn.execute(mod_ta_tf3)
conn.execute(update_ta_tf3)
conn.commit()

In [ ]:
%sql select * from ta_tf where rsi_indicator = 'overbought';

""


In [ ]:
ta_tf_df = pd.read_sql_query("select * from ta_tf", conn)
ta_tf_df

,ticker,date,rsi_value,afterHours,preMarket,open,high,low,close,adjclose,volume,closed_volatility,open_volatility,rsi_weighted,rsi_indicator
0,AAPL,2023-08-01,63.309440,195.30,196.35,196.240005,196.729996,195.279999,195.610001,194.576660,35175100,1.05,0.630005,2.226916e+09,neutral
1,GOOGL,2023-08-01,65.855486,130.50,132.55,130.779999,132.630005,130.679993,131.550003,131.399200,23166800,2.05,-0.770004,1.525661e+09,neutral
2,MSFT,2023-08-01,47.808717,334.39,335.53,335.190002,338.540009,333.700012,336.339996,333.729340,18311900,1.14,-1.149994,8.754684e+08,neutral
3,AMZN,2023-08-01,54.698728,131.26,133.57,133.550003,133.690002,131.619995,131.690002,131.690002,42098500,2.31,1.860001,2.302734e+09,neutral
4,TSLA,2023-08-01,49.163105,258.73,267.38,266.260010,266.470001,260.250000,261.070007,261.070007,83166000,8.65,5.190002,4.088699e+09,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,AMZN,2023-08-31,58.361861,138.40,135.12,135.059998,138.789993,135.000000,138.009995,138.009995,58781300,-3.28,-2.949997,3.430586e+09,neutral
180,TSLA,2023-08-31,57.915162,257.96,255.29,255.979996,261.179993,255.050003,258.079987,258.079987,108861700,-2.67,-2.099991,6.304743e+09,neutral
181,COST,2023-08-31,53.072991,549.99,542.22,540.000000,552.200012,540.000000,549.280029,534.438354,2086500,-7.77,-9.280029,1.107368e+08,neutral
182,MA,2023-08-31,67.687616,412.24,415.70,414.209991,416.290009,411.779999,412.640015,410.825256,2535100,3.46,1.569976,1.715949e+08,neutral


In [ ]:
ta_tf_df.to_csv('ta_tf.csv', index=False)
conn.close()

In [ ]:
!gcloud auth login

!gcloud config set project de-fall-24

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=04bJJl81qDSLLoSBWpJ0dkSOeOt1dD&prompt=consent&token_usage=remote&access_type=offline&code_challenge=pcxupR2Fr4yAmlG-F_Zm_txK6hBvwronmray3_GXa1k&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVG7fiRvVk6XJKebzT5XgQkUey35rqjXTekP54baN4Nx1jptKb_siLOr6k-ErFA01zQPvA

You are now logged in as [cjoyce764@gmail.com].
Your current project is

In [ ]:
!gsutil cp ta_tf.csv gs://cj_de/

Copying file://ta_tf.csv [Content-Type=text/csv]...
/ [1 files][ 35.8 KiB/ 35.8 KiB]                                                
Operation completed over 1 objects/35.8 KiB.                                     
